# IBKR Positions to Database - Streamlined
Minimal code version focused on efficient database writes

In [110]:
import time
import threading
import pandas as pd
import random
import datetime
import psycopg2
import os
import sys
import json
from decimal import Decimal
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [111]:
def safe_float_convert(value):
    """Safely convert any value to float, handling Decimal types"""
    if value is None or pd.isna(value):
        return None
    if isinstance(value, str):
        try:
            return float(value)
        except ValueError:
            return None
    if isinstance(value, Decimal):
        return float(value)
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

In [112]:
class IBKRPositionApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.positions = {}
        self.market_data = {}
        self.req_id = 1000
        self.position_data_received = False
        self.market_data_requests = {}
        self.account_updates = {}
        self.account_update_complete = False
        
    def connectTWS(self, port=7497):
        """Connect to TWS or IB Gateway"""
        try:
            client_id = random.randint(1, 999)
            import subprocess
            try:
                result = subprocess.run(['ip', 'route', 'show'], capture_output=True, text=True)
                host_ip = None
                for line in result.stdout.split('\n'):
                    if 'default via' in line:
                        host_ip = line.split()[2]
                        break
            except:
                host_ip = '127.0.0.1'
            
            self.connect(host_ip, port, client_id)
            thread = threading.Thread(target=self.run)
            thread.daemon = True
            thread.start()
            time.sleep(2)
            
            if self.isConnected():
                logger.info(f"Connected to TWS/Gateway on {host_ip}:{port}")
                return True
            return False
        except Exception as e:
            logger.error(f"Connection error: {e}")
            return False
    
    def position(self, account, contract, position, avgCost):
        """Callback for position data"""
        key = f"{contract.symbol}_{contract.secType}_{contract.strike}_{contract.right}_{contract.lastTradeDateOrContractMonth}"
        
        avg_cost_raw = safe_float_convert(avgCost)
        position_float = safe_float_convert(position)
        multiplier = 100.0 if contract.secType == 'OPT' else 1.0
        avg_cost_per_unit = (avg_cost_raw / multiplier) if avg_cost_raw is not None else None

        self.positions[key] = {
            'Account': account,
            'Symbol': contract.symbol,
            'SecType': contract.secType,
            'Description': f"{contract.symbol} {contract.lastTradeDateOrContractMonth} {contract.strike} {contract.right}",
            'AvgCost': avg_cost_per_unit,
            'Strike': safe_float_convert(contract.strike) if hasattr(contract, 'strike') else None,
            'Right': contract.right if hasattr(contract, 'right') else None,
            'Expiry': contract.lastTradeDateOrContractMonth,
            'Position': position_float,
            'Contract': contract,
            'ConId': contract.conId,
            'CurrentPrice': None,
            'MarketVal': None,
            'UnrealizedPnL': None
        }
    
    def positionEnd(self):
        """Callback when all positions have been received"""
        self.position_data_received = True
        logger.info("All position data received")
    
    def updateAccountValue(self, key, val, currency, accountName):
        """Callback for account value updates"""
        # Store account level data if needed
        pass
    
    def updatePortfolio(self, contract, position, marketPrice, marketValue, averageCost, 
                       unrealizedPNL, realizedPNL, accountName):
        """Callback for portfolio updates with market values"""
        key = f"{contract.symbol}_{contract.secType}_{contract.strike}_{contract.right}_{contract.lastTradeDateOrContractMonth}"
        
        print(f"Portfolio update for {key}:")
        print(f"  Position: {position}")
        print(f"  Market Price: {marketPrice}")
        print(f"  Market Value: {marketValue}")
        print(f"  Average Cost: {averageCost}")
        print(f"  Unrealized PnL: {unrealizedPNL}")
        
        # Store the portfolio update data
        self.account_updates[key] = {
            'MarketPrice': safe_float_convert(marketPrice),
            'MarketValue': safe_float_convert(marketValue),
            'UnrealizedPNL': safe_float_convert(unrealizedPNL),
            'AverageCost': safe_float_convert(averageCost),
            'Position': safe_float_convert(position)
        }
    
    def accountDownloadEnd(self, accountName):
        """Callback when account download is complete"""
        self.account_update_complete = True
        print(f"Account download complete for {accountName}")
    
    def error(self, reqId, errorCode, errorString):
        """Callback for error messages"""
        # Only print non-informational errors
        if errorCode not in [2104, 2106, 2158]:
            print(f"ERROR: reqId={reqId}, code={errorCode}, msg={errorString}")
    
    def get_positions_data(self):
        """Request positions and wait for data"""
        self.reqPositions()
        timeout = 30
        start_time = time.time()
        
        while not self.position_data_received and (time.time() - start_time) < timeout:
            time.sleep(0.1)
        
        return len(self.positions)
    
    def get_account_updates(self, account_id):
        """Request account updates to get market values"""
        print(f"\n=== Requesting account updates for {account_id} ===")
        self.reqAccountUpdates(True, account_id)
        
        # Wait for account updates
        timeout = 10
        start_time = time.time()
        while not self.account_update_complete and (time.time() - start_time) < timeout:
            time.sleep(0.1)
        
        # Stop account updates
        self.reqAccountUpdates(False, account_id)
        
        # Update positions with account data
        print(f"\n=== Updating positions with account data ===")
        for key, pos in self.positions.items():
            if key in self.account_updates:
                acc_data = self.account_updates[key]
                market_price = acc_data.get('MarketPrice')
                market_value = acc_data.get('MarketValue')
                unrealized_pnl = acc_data.get('UnrealizedPNL')
                
                # For options, market price needs to be per contract (not per share)
                if pos['SecType'] == 'OPT' and market_price is not None:
                    # IBKR gives market price per share, convert to per contract
                    self.positions[key]['CurrentPrice'] = market_price
                    self.positions[key]['MarketVal'] = market_value
                    self.positions[key]['UnrealizedPnL'] = unrealized_pnl
                    print(f"Updated {key}: Price={market_price}, MktVal={market_value}, PnL={unrealized_pnl}")
                else:
                    self.positions[key]['CurrentPrice'] = market_price
                    self.positions[key]['MarketVal'] = market_value
                    self.positions[key]['UnrealizedPnL'] = unrealized_pnl
                    print(f"Updated {key}: Price={market_price}, MktVal={market_value}, PnL={unrealized_pnl}")
            else:
                print(f"No account update for {key}")
    
    def find_vertical_spreads(self, df):
        """Identify vertical spreads from options positions"""
        options_df = df[df['SecType'] == 'OPT'].copy()
        if options_df.empty:
            return pd.DataFrame()
        
        spreads = []
        grouped = options_df.groupby(['Symbol', 'Right', 'Expiry'])
        
        for (symbol, right, expiry), group in grouped:
            if len(group) >= 2:
                for i in range(len(group)):
                    for j in range(i + 1, len(group)):
                        pos1, pos2 = group.iloc[i], group.iloc[j]
                        
                        strike1 = safe_float_convert(pos1['Strike'])
                        strike2 = safe_float_convert(pos2['Strike'])
                        position1 = safe_float_convert(pos1['Position'])
                        position2 = safe_float_convert(pos2['Position'])
                        
                        if all(v is not None for v in [strike1, strike2, position1, position2]):
                            if (position1 > 0 and position2 < 0) or (position1 < 0 and position2 > 0):
                                if abs(position1) == abs(position2):
                                    spread_type = "Bull" if (strike1 < strike2 and position1 > 0) else "Bear"
                                    spread_type += " Call" if right == "C" else " Put"
                                    
                                    avg_cost1 = safe_float_convert(pos1['AvgCost'])
                                    avg_cost2 = safe_float_convert(pos2['AvgCost'])
                                    net_cost = (avg_cost1 * position1 + avg_cost2 * position2) / abs(position1)
                                    
                                    current1 = safe_float_convert(pos1['CurrentPrice'])
                                    current2 = safe_float_convert(pos2['CurrentPrice'])
                                    current_value = None
                                    market_val = None
                                    unrealized_pnl = None
                                    
                                    if current1 is not None and current2 is not None and current1 > 0 and current2 > 0:
                                        current_value = (current1 * position1 + current2 * position2) / abs(position1)
                                        market_val = current_value * abs(position1) * 100
                                        if net_cost is not None:
                                            unrealized_pnl = (current_value - net_cost) * abs(position1) * 100
                                    
                                    spreads.append({
                                        'Symbol': symbol,
                                        'Description': f"{spread_type} {strike1}/{strike2} {expiry}",
                                        'AvgCost': net_cost,
                                        'CurrentPrice': current_value,
                                        'Position': abs(position1),
                                        'MarketVal': market_val,
                                        'UnrealizedPnL': unrealized_pnl
                                    })
        
        return pd.DataFrame(spreads)
    
    def get_positions_dataframe(self):
        """Convert positions to DataFrame"""
        data = []
        for key, pos in self.positions.items():
            data.append({
                'Symbol': pos['Symbol'],
                'SecType': pos['SecType'],
                'Description': pos['Description'],
                'AvgCost': pos['AvgCost'],
                'CurrentPrice': pos['CurrentPrice'],
                'Position': pos['Position'],
                'MarketVal': pos['MarketVal'],
                'UnrealizedPnL': pos['UnrealizedPnL'],
                'Strike': pos['Strike'],
                'Right': pos['Right'],
                'Expiry': pos['Expiry']
            })
        return pd.DataFrame(data)
    
    def disconnect_tws(self):
        """Disconnect from TWS"""
        if self.isConnected():
            self.disconnect()

In [113]:
project_root = os.getcwd()
config_path = os.path.join(project_root, 'config')
sys.path.insert(0, config_path)

credentials_file = os.path.join(config_path, 'credentials.json')
with open(credentials_file, 'r') as f:
    creds = json.load(f)

pg_creds = creds['database']['postgresql']
pg_creds['host'] = '127.0.0.1'
pg_creds['port'] = 5433

In [114]:
def get_option_strategies():
    """Get option strategies from the database"""
    try:
        conn = psycopg2.connect(
            host=pg_creds['host'],
            port=pg_creds['port'],
            database=pg_creds['database'],
            user=pg_creds['user'],
            password=pg_creds['password']
        )
        
        query = """
        SELECT id, strategy_type, ticker, trigger_price, strike_buy, strike_sell, 
               estimated_premium, options_expiry_date, scrape_date, strategy_status, trade_id
        FROM option_strategies
        WHERE strategy_status IN ('order placed', 'active')
        ORDER BY scrape_date DESC
        """
        
        db_strategies_df = pd.read_sql_query(query, conn)
        conn.close()
        return db_strategies_df
        
    except Exception as e:
        print(f"Database query failed: {e}")
        return pd.DataFrame()

In [115]:
def join_spreads_with_database(spreads_df, db_strategies_df):
    """Join IBKR spreads with database strategies"""
    if spreads_df.empty or db_strategies_df.empty:
        return pd.DataFrame()
    
    joined_data = []
    unmatched_spreads = []
    
    for _, ibkr_row in spreads_df.iterrows():
        description = ibkr_row['Description']
        symbol = ibkr_row['Symbol']
        
        parts = description.split()
        strategy_type = ' '.join(parts[:2])
        strike_info = parts[2]
        expiry = parts[3]
        
        strikes = strike_info.split('/')
        strike1, strike2 = float(strikes[0]), float(strikes[1])
        
        if "Bull" in strategy_type:
            db_strike_buy = min(strike1, strike2)
            db_strike_sell = max(strike1, strike2)
        else:
            db_strike_buy = max(strike1, strike2)
            db_strike_sell = min(strike1, strike2)
        
        expiry_date = f"{expiry[:4]}-{expiry[4:6]}-{expiry[6:]}"
        
        matches = db_strategies_df[
            (db_strategies_df['ticker'] == symbol) &
            (db_strategies_df['strategy_type'] == strategy_type) &
            (db_strategies_df['strike_buy'] == db_strike_buy) &
            (db_strategies_df['strike_sell'] == db_strike_sell) &
            (db_strategies_df['options_expiry_date'].astype(str) == expiry_date)
        ]
        
        if matches.empty:
            unmatched_spreads.append(f"{symbol} {description}")
        
        for _, db_row in matches.iterrows():
            joined_data.append({
                'ibkr_symbol': symbol,
                'ibkr_description': description,
                'ibkr_avg_cost': ibkr_row['AvgCost'],
                'ibkr_current_price': ibkr_row['CurrentPrice'],
                'ibkr_unrealized_pnl': ibkr_row['UnrealizedPnL'],
                'ibkr_market_val': ibkr_row['MarketVal'],
                'ibkr_position': ibkr_row['Position'],
                'db_id': db_row['id'],
                'db_ticker': db_row['ticker'],
                'db_strategy_type': db_row['strategy_type'],
                'db_estimated_premium': db_row['estimated_premium'],
                'db_trade_id': db_row['trade_id'],
                'premium_difference': ibkr_row['AvgCost'] - db_row['estimated_premium']
            })
    
    # Print unmatched spreads
    if unmatched_spreads:
        print("\n" + "="*80)
        print("⚠️  SPREADS FOUND IN IBKR BUT NOT IN DATABASE:")
        print("="*80)
        for spread in unmatched_spreads:
            print(f"   - {spread}")
        print("="*80 + "\n")
    
    return pd.DataFrame(joined_data)

In [116]:
def insert_positions_to_database(joined_df):
    """Insert joined position data into ibkr_positions table"""
    if joined_df.empty:
        return False
    
    try:
        conn = psycopg2.connect(
            host=pg_creds['host'],
            port=pg_creds['port'],
            database=pg_creds['database'],
            user=pg_creds['user'],
            password=pg_creds['password']
        )
        
        cursor = conn.cursor()
        
        for _, row in joined_df.iterrows():
            insert_sql = """
            INSERT INTO ibkr_positions (
                ibkr_symbol, ibkr_description, ibkr_avg_cost, ibkr_current_price,
                ibkr_unrealized_pnl, ibkr_market_val, ibkr_position,
                db_id, db_ticker, db_strategy_type, db_estimated_premium,
                db_trade_id, premium_difference
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (ibkr_symbol, ibkr_description, db_id)
            DO UPDATE SET
                ibkr_current_price = EXCLUDED.ibkr_current_price,
                ibkr_unrealized_pnl = EXCLUDED.ibkr_unrealized_pnl,
                ibkr_market_val = EXCLUDED.ibkr_market_val,
                updated_at = CURRENT_TIMESTAMP
            """
            
            cursor.execute(insert_sql, (
                row['ibkr_symbol'],
                row['ibkr_description'],
                row['ibkr_avg_cost'],
                row['ibkr_current_price'],
                row['ibkr_unrealized_pnl'],
                row['ibkr_market_val'],
                row['ibkr_position'],
                row['db_id'],
                row['db_ticker'],
                row['db_strategy_type'],
                row['db_estimated_premium'],
                row['db_trade_id'],
                row['premium_difference']
            ))
        
        conn.commit()
        cursor.close()
        conn.close()
        return True
        
    except Exception as e:
        print(f"Database insert failed: {e}")
        return False

In [117]:
def capture_market_snapshots(positions_df, spreads_df, joined_df, pg_creds):
    """Capture market snapshots from IBKR data to database"""
    snapshots_created = 0
    snapshots_failed = 0

    print("\n" + "="*80)
    print("📸 CAPTURING MARKET SNAPSHOTS")
    print("="*80)

    for _, spread_row in joined_df.iterrows():
        db_trade_id = spread_row['db_trade_id']

        # Get position_id from database
        try:
            conn = psycopg2.connect(
                host=pg_creds['host'],
                port=pg_creds['port'],
                database=pg_creds['database'],
                user=pg_creds['user'],
                password=pg_creds['password']
            )
            cursor = conn.cursor()
            cursor.execute("SELECT id FROM ibkr_positions WHERE db_trade_id = %s LIMIT 1", (db_trade_id,))
            result = cursor.fetchone()
            cursor.close()
            conn.close()

            if not result:
                print(f"⚠️  No position found for {db_trade_id}")
                snapshots_failed += 1
                continue
            position_id = str(result[0])
        except Exception as e:
            print(f"❌ Error: {e}")
            snapshots_failed += 1
            continue

        # Parse spread to find legs
        description = spread_row['ibkr_description']
        symbol = spread_row['ibkr_symbol']
        parts = description.split()
        strike_info = parts[2]
        expiry = parts[3]
        strikes = strike_info.split('/')
        strike1, strike2 = float(strikes[0]), float(strikes[1])
        right = 'P' if 'Put' in description else 'C'

        # Find legs in positions_df
        leg1 = positions_df[
            (positions_df['Symbol'] == symbol) &
            (positions_df['Strike'] == strike1) &
            (positions_df['Right'] == right) &
            (positions_df['Expiry'] == expiry)
        ]
        leg2 = positions_df[
            (positions_df['Symbol'] == symbol) &
            (positions_df['Strike'] == strike2) &
            (positions_df['Right'] == right) &
            (positions_df['Expiry'] == expiry)
        ]

        if leg1.empty or leg2.empty:
            print(f"⚠️  Missing legs for {description}")
            snapshots_failed += 1
            continue

        leg1_row, leg2_row = leg1.iloc[0], leg2.iloc[0]

        # Insert snapshot - convert numpy types to Python native types using safe_float_convert
        try:
            conn = psycopg2.connect(
                host=pg_creds['host'],
                port=pg_creds['port'],
                database=pg_creds['database'],
                user=pg_creds['user'],
                password=pg_creds['password']
            )
            cursor = conn.cursor()
            cursor.execute("""
                INSERT INTO market_snapshots (
                    position_id, db_trade_id,
                    spread_market_val, spread_unrealized_pnl, spread_current_price,
                    leg1_symbol, leg1_description, leg1_market_val, leg1_unrealized_pnl,
                    leg1_current_price, leg1_position,
                    leg2_symbol, leg2_description, leg2_market_val, leg2_unrealized_pnl,
                    leg2_current_price, leg2_position
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                position_id, 
                db_trade_id,
                safe_float_convert(spread_row['ibkr_market_val']), 
                safe_float_convert(spread_row['ibkr_unrealized_pnl']), 
                safe_float_convert(spread_row['ibkr_current_price']),
                str(leg1_row['Symbol']), 
                str(leg1_row['Description']), 
                safe_float_convert(leg1_row['MarketVal']),
                safe_float_convert(leg1_row['UnrealizedPnL']), 
                safe_float_convert(leg1_row['CurrentPrice']), 
                safe_float_convert(leg1_row['Position']),
                str(leg2_row['Symbol']), 
                str(leg2_row['Description']), 
                safe_float_convert(leg2_row['MarketVal']),
                safe_float_convert(leg2_row['UnrealizedPnL']), 
                safe_float_convert(leg2_row['CurrentPrice']), 
                safe_float_convert(leg2_row['Position'])
            ))
            conn.commit()
            cursor.close()
            conn.close()
            snapshots_created += 1
            print(f"✅ {symbol} {description}")
        except Exception as e:
            print(f"❌ {symbol}: {e}")
            snapshots_failed += 1

    print("="*80)
    print(f"📊 SUMMARY: {snapshots_created} created, {snapshots_failed} failed")
    print("="*80 + "\n")
    return snapshots_created

In [118]:
app = IBKRPositionApp()
connected = app.connectTWS()

if connected:
    # Get positions
    app.get_positions_data()
    
    # Get account updates which includes market values and prices
    account_id = "DU9233079"  # Your paper trading account
    app.get_account_updates(account_id)
    
    # Get dataframes
    positions_df = app.get_positions_dataframe()
    spreads_df = app.find_vertical_spreads(positions_df)
    
    # Get database strategies and join
    db_strategies_df = get_option_strategies()
    
    if not spreads_df.empty and not db_strategies_df.empty:
        joined_df = join_spreads_with_database(spreads_df, db_strategies_df)
        if not joined_df.empty:
            success = insert_positions_to_database(joined_df)
            print(f"Inserted {len(joined_df)} positions to database")
            
            # Capture market snapshots
            snapshots_created = capture_market_snapshots(positions_df, spreads_df, joined_df, pg_creds)
            print(f"Captured {snapshots_created} market snapshots")
    
    app.disconnect_tws()
else:
    print("Failed to connect to TWS/Gateway")

INFO:ibapi.client:sent startApi
INFO:ibapi.client:REQUEST startApi {}
INFO:ibapi.client:SENDING startApi b'\x00\x00\x00\n71\x002\x00664\x00\x00'
INFO:ibapi.wrapper:ANSWER connectAck {}
INFO:ibapi.wrapper:ANSWER nextValidId {'orderId': 1}
INFO:ibapi.wrapper:ANSWER managedAccounts {'accountsList': 'DU9233079'}
INFO:__main__:Connected to TWS/Gateway on 172.21.240.1:7497
INFO:ibapi.client:REQUEST reqPositions {}
INFO:ibapi.client:SENDING reqPositions b'\x00\x00\x00\x0561\x001\x00'
INFO:__main__:All position data received
INFO:ibapi.client:REQUEST reqAccountUpdates {'subscribe': True, 'acctCode': 'DU9233079'}
INFO:ibapi.client:SENDING reqAccountUpdates b'\x00\x00\x00\x106\x002\x001\x00DU9233079\x00'
INFO:ibapi.wrapper:ANSWER updateAccountTime {'timeStamp': '07:09'}
INFO:ibapi.wrapper:ANSWER updateAccountTime {'timeStamp': '07:09'}
INFO:ibapi.wrapper:ANSWER updateAccountTime {'timeStamp': '07:09'}
INFO:ibapi.wrapper:ANSWER updateAccountTime {'timeStamp': '07:09'}
INFO:ibapi.wrapper:ANSWER up


=== Requesting account updates for DU9233079 ===
Portfolio update for AMZN_OPT_195.0_P_20251031:
  Position: 1.0
  Market Price: 0.3674817
  Market Value: 36.75
  Average Cost: 355.7959
  Unrealized PnL: -319.05
Portfolio update for AMZN_OPT_205.0_P_20251031:
  Position: -1.0
  Market Price: 1.1603284
  Market Value: -116.03
  Average Cost: 619.2013
  Unrealized PnL: 503.17
Portfolio update for IWM_OPT_260.0_C_20251031:
  Position: -1.0
  Market Price: 0.2114646
  Market Value: -21.15
  Average Cost: 204.3713
  Unrealized PnL: 183.22
Portfolio update for IWM_OPT_270.0_C_20251031:
  Position: 1.0
  Market Price: 0.02284365
  Market Value: 2.28
  Average Cost: 45.6259
  Unrealized PnL: -43.34
Portfolio update for META_OPT_630.0_P_20251024:
  Position: 0.0
  Market Price: 0.0
  Market Value: 0.0
  Average Cost: 0.0
  Unrealized PnL: 0.0
Portfolio update for META_OPT_640.0_P_20251024:
  Position: 0.0
  Market Price: 0.0
  Market Value: 0.0
  Average Cost: 0.0
  Unrealized PnL: 0.0
Portfol

/tmp/ipykernel_397058/2637932311.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  db_strategies_df = pd.read_sql_query(query, conn)



⚠️  SPREADS FOUND IN IBKR BUT NOT IN DATABASE:
   - QQQ Bear Put 566.0/556.0 20251031

Inserted 4 positions to database

📸 CAPTURING MARKET SNAPSHOTS
✅ AMZN Bull Put 195.0/205.0 20251031
✅ IWM Bear Call 260.0/270.0 20251031
✅ NKE Bull Put 55.0/67.0 20251031


INFO:ibapi.client:disconnecting
INFO:ibapi.wrapper:ANSWER connectionClosed {}


✅ SPY Bull Put 620.0/629.0 20251107
📊 SUMMARY: 4 created, 0 failed

Captured 4 market snapshots


In [119]:
print("Positions DataFrame:")
positions_df


Positions DataFrame:


,Symbol,SecType,Description,AvgCost,CurrentPrice,Position,MarketVal,UnrealizedPnL,Strike,Right,Expiry
0,AAPL,OPT,AAPL 20251031 230.0 P,0.000000,NaN,0.0,NaN,NaN,230.0,P,20251031
1,AAPL,OPT,AAPL 20251031 240.0 P,0.000000,NaN,0.0,NaN,NaN,240.0,P,20251031
2,UNH,OPT,UNH 20251024 390.0 C,0.000000,0.000000,0.0,0.00,0.00,390.0,C,20251024
3,NKE,OPT,NKE 20251024 60.0 P,0.000000,0.000000,0.0,0.00,0.00,60.0,P,20251024
4,IWM,OPT,IWM 20251031 260.0 C,2.043713,0.211465,-1.0,-21.15,183.22,260.0,C,20251031
5,QQQ,OPT,QQQ 20251031 566.0 P,5.269488,0.300874,-1.0,-30.09,496.86,566.0,P,20251031
6,NKE,OPT,NKE 20251031 55.0 P,0.046259,0.007834,1.0,0.78,-3.84,55.0,P,20251031
7,SPY,OPT,SPY 20251107 620.0 P,4.266259,0.394573,1.0,39.46,-387.17,620.0,P,20251107
8,QQQ,OPT,QQQ 20251031 556.0 P,3.890484,0.202230,1.0,20.22,-368.83,556.0,P,20251031
9,AMZN,OPT,AMZN 20251031 195.0 P,3.557959,0.367482,1.0,36.75,-319.05,195.0,P,20251031


In [120]:
print("\nSpreads DataFrame:")
spreads_df


Spreads DataFrame:


,Symbol,Description,AvgCost,CurrentPrice,Position,MarketVal,UnrealizedPnL
0,AMZN,Bull Put 195.0/205.0 20251031,-2.634054,-0.792847,1.0,-79.284670,184.120730
1,IWM,Bear Call 260.0/270.0 20251031,-1.587454,-0.188621,1.0,-18.862095,139.883305
2,NKE,Bull Put 55.0/67.0 20251031,-1.237454,-0.379476,1.0,-37.947615,85.797785
3,QQQ,Bear Put 566.0/556.0 20251031,-1.379004,-0.098644,1.0,-9.864425,128.035975
4,SPY,Bull Put 620.0/629.0 20251107,-1.187454,-0.130635,1.0,-13.063495,105.681905


In [121]:
db_strategies_df

,id,strategy_type,ticker,trigger_price,strike_buy,strike_sell,estimated_premium,options_expiry_date,scrape_date,strategy_status,trade_id
0,861,Bear Call,IWM,251.76,270.0,260.0,161.0,2025-10-31,2025-10-15T11:02:58.174561,order placed,funny-tangerine-serval
1,858,Bull Put,SPY,655.16,620.0,629.0,84.0,2025-11-07,2025-10-14T11:04:03.405443,order placed,slug-of-incredible-prosperity
2,854,Bull Put,QQQ,591.1,556.0,566.0,140.0,2025-10-31,2025-10-14T11:03:55.799491,order placed,crystal-hippo-of-warranty
3,851,Bull Put,AMZN,214.12,195.0,205.0,240.0,2025-10-31,2025-10-14T11:03:48.451409,order placed,intrepid-serious-pillbug
4,839,Bull Put,MSFT,511.78,475.0,485.0,191.0,2025-10-31,2025-10-10T11:03:57.147057,order placed,independent-hippo-of-promise
...,...,...,...,...,...,...,...,...,...,...,...
69,39,Bull Put,META,508.07,455.0,465.0,230.0,2025-05-09,2025-04-16T07:52:15.912922,order placed,peach-swan-of-variation
70,28,Bull Put,META,518.65,465.0,475.0,273.0,2025-05-09,2025-04-15T04:58:35.221500,order placed,lyrical-inquisitive-chachalaca
71,10,Bear Call,SPY,537.02,570.0,560.0,190.0,2025-04-25,2025-04-14T07:32:32.610704,order placed,caped-robin-of-chemistry
72,9,Bear Call,GOOGL,159,185.0,175.0,10.0,2025-04-25,2025-04-14T07:32:32.512393,order placed,daffy-hedgehog-of-vigor
